In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
import bitsandbytes as bnb
import os
#import wandb
from datetime import datetime

from transformers import PreTrainedTokenizerFast, AutoTokenizer, AdamW, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split

#os.environ["TOKENIZERS_PARALLELISM"] = "true" 
#torch.backends.cuda.matmul.allow_tf32=True
#torch.set_float32_matmul_precision('medium')
#torch.backends.cudnn.benchmark = True

In [2]:
cfg_random_seed = 2024
cfg_max_length = 128
cfg_max_token = 128
cfg_full_name = 'QLoRA + 4bit quantization + LDCC-SOLAR-10.7B'

cfg = {
    'random_seed' : 2024,
    'max_length' : 128,
    'max_token' : 128,
    'full_name' : 'QLoRA + 4bit quantization + LDCC-SOLAR-10.7B',
    'train_size' : 0.9,
    'test_size' : 0.1,
    'do_eval' : True,
    'eval_strategy' : 'epoch' # 'epoch', 'steps' , 'no',
}


# Data Preprocessing

In [3]:
# 데이터 로드 
data = pd.read_csv('./data/train_data_final.csv')

# 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('LDCC/LDCC-SOLAR-10.7B',
                                                    eos_token='</s>')

# 데이터 포맷팅 및 토큰화
formatted_data = []
for _, row in tqdm(data.iterrows()):
    input_text = row['질문'] + tokenizer.eos_token + row['답변']
    input_enc = tokenizer.encode(input_text, return_tensors='pt',
                                 padding='max_length',
                                 truncation=True,
                                 max_length=cfg_max_length)
    formatted_data.append(input_enc)

print('Done.')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
6440it [00:02, 2151.90it/s]

Done.


In [4]:
# train_test_split
train_data, valid_data = train_test_split(formatted_data, 
                                          train_size=cfg['train_size'], test_size=cfg['test_size'],
                                          shuffle=True, 
                                          random_state=cfg['random_seed']
                                         )

In [5]:
print(len(train_data))
print(len(valid_data))

5796
644


In [6]:
#formatted_data = torch.cat(formatted_data, dim=0)
formatted_train_data = torch.cat(train_data, dim=0)
formatted_valid_data = torch.cat(valid_data, dim=0)

In [7]:
# 모델 로드

model_id = "LDCC/LDCC-SOLAR-10.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map='balanced', # 'balanced' 로 수정
                                             # torch_dtype=torch.float32,
                                             )


model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    #target_modules=["query_key_value"], 
    target_modules=[
    "q_proj",
    "up_proj",
    "o_proj",
    "k_proj",
    "down_proj",
    "gate_proj",
    "v_proj"],
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

trainable params: 31457280 || all params: 5659561984 || trainable%: 0.555825346359525


In [8]:
# eval_dataset 추가
trainer = transformers.Trainer(
    model=model,
    
    train_dataset=formatted_train_data,
    
    # evaluation arguments 추가 
    eval_dataset=formatted_valid_data,
    
    args=transformers.TrainingArguments(
        per_device_train_batch_size=128, # 128도 가능할 듯?

        # evaluation args 추가
        do_eval=cfg['do_eval'],
        per_device_eval_batch_size=16, 
        evaluation_strategy = cfg['eval_strategy'],
    
        gradient_accumulation_steps=1,
        # max_steps=50, # default=1
        num_train_epochs=1, # default : 1
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        # logging_dir = './log/logs' 
        output_dir = "outputs",
        optim="paged_adamw_8bit" #  defaults to "adamw_torch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False 


In [9]:
print('Start train!')
print(f'Start time : {datetime.now()}')
trainer.train()
print('End train!')
print(f'End time : {datetime.now()}')

Start train!
Start time : 2024-03-06 06:22:50.539294


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.916900,0.891262


End train!
End time : 2024-03-06 07:06:13.779540


In [10]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(48000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [13]:
# 모델 저장 
model.save_pretrained('./outputs', save_peft_format=True)

# Inference code
https://dacon.io/competitions/official/236216/codeshare/9690?page=1&dtype=recent

https://www.datacamp.com/tutorial/solar-10-7b-fine-tuned-model-tutorial

In [14]:
import time
import datetime
from huggingface_hub import hf_hub_download
import pandas as pd
import numpy as np
from peft import PeftModel, PeftConfig

In [16]:
# peft_model_id = "LDCC/LDCC-SOLAR-10.7B"
# config = PeftConfig.from_pretrained(peft_model_id)  # 
tokenizer = PreTrainedTokenizerFast.from_pretrained('LDCC/LDCC-SOLAR-10.7B',
                                                    eos_token='</s>',
                                                   device_map='balanced')

# saved_model = AutoModelForCausalLM.from_pretrained('./outputs',
#                                                   device_map='balanced')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
#model.eval()
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(48000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_la

In [ ]:
test = pd.read_csv('./data/test.csv')

preds = []
cfg_max_token = 128

for test_q in tqdm(test['질문']):
    # 입력 텍스트를 토큰화 하고 모델 입력 형태로 변환
    input_ = tokenizer(test_q + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    outputs = model.generate(input_.input_ids.to(model.device), max_new_tokens=cfg_max_token)

    # 생성된 텍스트(답변) 저장
    full_text = tokenizer.batch_decode(outputs.detach().cpu(), skip_special_tokens=False)
    # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
    full_text = full_text[0]
    answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
    answer_only = full_text[answer_start:].strip()
    answer_only = answer_only.replace('\n', ' ')
    preds.append(answer_only)

  9%|▉         | 12/130 [05:28<53:48, 27.36s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


# Submission

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
modelEmb = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = modelEmb.encode(preds)
pred_embeddings.shape

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
#submit.head()

In [ ]:
# 리더보드 제출을 위한 csv파일 생성

submit.to_csv(f'./submissions/{cfg['full_name']}_submit3.csv', index=False)